In [60]:
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import numpy as np
from scipy.spatial import distance

def fixed_xor(a, b):
    return bytes([_a ^ _b for _a, _b in zip(a, b)])

B = lambda s: [s[i:i+16] for i in range(0, len(s), 16)]

def keystream(key, nonce, cnt = 0, end = 2**64):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()
    while cnt < end:
        yield aes.update(nonce + cnt.to_bytes(length=64//8, byteorder='little'))
        cnt += 1

def aes_ctr(ciphertxt, key, nonce):
    return b''.join([fixed_xor(k, c) for k, c in zip(keystream(key, nonce), B(ciphertxt))])

with open('t8.shakespeare.txt', 'r') as f:
    shks = np.frombuffer(f.read().encode("ASCII"), 'u1')

ascii_hist = lambda s: np.histogram(s, bins=range(2**8), density=True)[0]

shks_hist = ascii_hist(shks)

score = lambda buf: distance.jensenshannon(shks_hist, ascii_hist(np.frombuffer(buf, 'u1')))

i2b = lambda i: bytes([i])

def break_sb_xor_key(cipher):
    cipherd = np.frombuffer(cipher, 'u1')
    evaluation = np.array([score(fixed_xor(cipher, i2b(i) * len(cipher))) for i in range(2**8)])
    return i2b(np.argmin(evaluation))

def rk_xor_key(plain, k):
    N = len(plain)
    n = len(k)
    return (k * np.ceil(N / n).astype(int))[:N]

def vigenere(plain, k):
    return fixed_xor(plain, rk_xor_key(plain, k))

hamming_distance = lambda x, y: int.from_bytes(fixed_xor(x, y), byteorder='big').bit_count()

def hamming_score(k_sz, cipher, n = 1):
    d = 0
    for i in range(n):
        d = d + hamming_distance(cipher[i*k_sz:(i+1)*k_sz], cipher[(i+1)*k_sz:(i+2)*k_sz]) / k_sz
        d /= 2
    return d

def break_vigenere(cipher, KEYSIZE):

    cipherd = np.frombuffer(cipher, 'u1')

    cipher_slices = np.pad(cipherd, (0, int(np.ceil(cipherd.shape[0] / KEYSIZE) * KEYSIZE - cipherd.shape[0])), mode='constant', constant_values=0).reshape(-1, KEYSIZE)
    key_candidate = b''.join(break_sb_xor_key(cipher_slices.T[i].tobytes()) for i in range(KEYSIZE))

    return key_candidate

In [61]:
with open("19.txt", "rb") as f:
    plaintxts = [base64.decodebytes(line) for line in f.readlines()]

In [62]:
K = np.random.bytes(16)

In [63]:
ciphertxts = [aes_ctr(plaintxt, K, b'\x00' * 16) for plaintxt in plaintxts]

In [64]:
m = len(min(ciphertxts, key=lambda x: len(x)))
truncated_ciphertxts = [ciphertxt[:m] for ciphertxt in ciphertxts]
concatenated_truncated_ciphertxts = b''.join(truncated_ciphertxts)

In [65]:
keystrim = break_vigenere(concatenated_truncated_ciphertxts, m)
keystrim

b"\x8a\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L\xaa\x87\x80\x04"

In [68]:
vigenere(truncated_ciphertxts[0], keystrim)

b'i have met them at c'

In [66]:
list(keystream(K, b'\x00' * 16, end=8))

[b"\xaa\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L",
 b"\xaa\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L\x84\xbe\xc4\xc2\x9e1='\x0c\xc4\x924\\\xc9\x95z",
 b"\xaa\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L",
 b'\x8e)\xbf\xc0\xb1g4l`\x01"\x08\xe7\xbc\xcdx\x01\xdb\x96\xe0\xbe\x15}|\xf9\xfe\xa2\xcaZ\xe4\x12\xe5',
 b"\xaa\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L",
 b'\xac\xf4\xe4\xc0KQ\x16\xd7\x8d<\xda\xeb\xe1\x8bpc\x01f[r\xc2F\x9cXQxW\x0c5}\xec\xe9',
 b"\xaa\x87\x80\x045$\x12e\xbe'\xff\x98\x0c)\xc8L",
 b"'.\xc0\xdc\xf9F\xd1\xfd\x80Vj\xaf6\x84\xb5\xa4\x140i^m\x87mGS\x1d\x8bH!\xd3\x08\xde"]

In [67]:
vigenere(next(keystream(K, b'\x00' * 16, cnt=0)), truncated_ciphertxts[0])

b'I have met them '

In [69]:
with open("20.txt", "rb") as f:
    plaintxts = [base64.decodebytes(line) for line in f.readlines()]

K = np.random.bytes(16)

ciphertxts = [aes_ctr(plaintxt, K, b'\x00' * 16) for plaintxt in plaintxts]

m = len(min(ciphertxts, key=lambda x: len(x)))

truncated_ciphertxts = [ciphertxt[:m] for ciphertxt in ciphertxts]

concatenated_truncated_ciphertxts = b''.join(truncated_ciphertxts)

keystrim = break_vigenere(concatenated_truncated_ciphertxts, m)

plaintxts = [vigenere(ciphertxt, keystrim) for ciphertxt in ciphertxts]

for p in plaintxts:
    print(p)

b'i\'m rated "R"...this is a warning, ya better void / P\xa9\xab\xef\x98\x88\xef\xb4\xca\xfd\x92\xf7\x81m\x10\xa3\xe7e=\x88\x14\x14\x1b\xd8\xa1\x9eg\x80\x91\x97d\xc6\xd8\xbf'
b'cuz I came back to attack others in spite- / Strike l\xaf\xa5\xfe\xcb\xc4\xe7\xa1\xc7\xa9\x8c\xff\x9d+R\xec\xc7u6\xdbp/I\xc2\xf5\xbfj\x95\x97\x8cl\xd7\xc9\xbeiO\xd3\xde\x16'
b"but don't be afraid in the dark, in a park / Not a sc\xb4\xab\xfa\x86\x88\xe1\xb4\x8f\xbc\xc2\xf5\x81uR\xec\xe1s1\xc9p<]\xd9\xea\xf6j\x9e\x8a\x97n\x9f\xd1\xb2lC\x9d\x98\x17\xe5\xb12$\r\xc8"
b"ya tremble like a alcoholic, muscles tighten up / Wha\xb2\xe9\xe8\xcb\xdc\xe6\xa7\xdb\xf1\xc2\xfa\x9ak\x16\xb8\xebo1\xdd \x7f\x1c\xf2\xee\xafj\x80\x80\x80+\xde\x9d\xa8nA\xd5\x8d\x17\xf4\xb4'"
b'suddenly you feel like your in a horror flick / You g\xb4\xaf\xf9\xcb\xd1\xe1\xb3\xdd\xfd\x8a\xf3\x92~\n\xec\xfait\xc6p)U\xd8\xe9\xfa,\x9c\x97\xc5\x7f\xd0\xd0\xb4uT\xd2\x8e\x17\xe7\xb4:5\r\xd2'
b"music's the clue, when I come your warned / Apocalyps\xa3\xee\xd5